# Chat Completion API

In [6]:
!pip install openai

## 환경설정
1. oepnai사이트에서 API KEY 발급
gogole colab - secret탭에 'OPENAI_API_KEY' 등록

In [23]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [24]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [26]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕~ 내 이름은 차은우야~"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [27]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕, 차은우! 만나서 반가워! 어떻게 지내고 있어? 도움이 필요하면 언제든지 말해줘!'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/
💛💛 무조건 암기
1. Instruction  지시사항
2. Context 문맥
3. Input Data/Example 입력/예시
4. Output Indicator 출력지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스AFP 속보시스템에서 도입되어 사용

In [28]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 교정해주세요.

제목: {title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: 졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환
- 교정 제목: 과중한 노동으로 끼니를 거르는 노동자들의 애환


In [29]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)

- 원래 제목: 주말 미친 폭우 예상, 모두들 무사하시길~
- 교정 제목: "주말 폭우 예보, 안전에 유의하세요"


## 영문텍스트에 anki단어장 만들기

## 연애코치 ReAct

In [30]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해 주세요.

    << Output Format >>
    1. 상황분석:

    2. 행동계획:

    3. 실행:
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [32]:
print(dating_coach('저 이번주말에 남자친구와 100일인데, 어떡하죠?'))

1. 상황분석:
   - 100일이라는 기념일은 커플에게 특별한 의미를 가집니다. 남자친구와의 관계가 이렇게 긴 시간을 지속해왔다는 것은 서로에 대한 이해와 사랑이 깊어졌다는 뜻입니다. 이 특별한 날을 기념하기 위해 특별한 계획이나 이벤트를 마련하고 싶다는 마음이 드는 것은 자연스러운 일입니다. 하지만 어떤 방식으로 기념할지 고민하고 있는 상황으로 보입니다.

2. 행동계획:
   - 기념일을 기념하기 위해 몇 가지 아이디어를 생각해볼 수 있습니다. 
     1. 특별한 데이트: 둘만의 아지트나 의미가 있는 장소에서 데이트를 계획해보세요. 예를 들어, 첫 데이트 장소나 서로에게 추억이 있는 곳 등.
     2. 선물 준비: 손편지나 그 동안의 추억이 담긴 사진 앨범을 만들어 선물하는 것도 좋습니다. 진솔한 마음이 담긴 선물은 남자친구에게 큰 감동을 줄 수 있을 것입니다.
     3. 특별한 음식: 둘이 함께 요리를 하거나 좋아하는 레스토랑에서 맛있는 음식을 즐기는 것도 좋은 방법입니다.
     4. 이벤트: 소규모 이벤트를 계획하여 서로의 관계를 더욱 돈독히 할 수 있는 기회를 만들어 보세요.

3. 실행:
   - 계획이 정해졌다면 남자친구와의 100일을 준비하는 전 과정을 실행해보세요. 미리 장소나 음식, 선물 등을 준비해 두는 것이 좋습니다. 기념일 당일에는 남자친구에게 서프라이즈를 해주거나 진솔하게 마음을 전해보세요. 정성스러운 준비와 함께 진솔한 대화가 이루어진다면 더욱 의미 있는 100일이 될 것입니다.


In [33]:
print(dating_coach('제가 어제 친구들이 놀다가 여자친구랑 연락을 잘 못해서, 단단히 삐져있는 상황입니다. 이 위기를 어떻게 극복해야 할까요?'))

1. 상황분석:
   현재 당신은 여자친구와의 연락 부재로 인해 그녀가 불편함을 느끼고 있고, 그로 인해 감정적으로 소외감을 느끼고 있습니다. 친구들과의 시간과 여자친구와의 소통 사이에서 균형을 잃은 것으로 보입니다. 이런 상황은 감정적인 오해를 불러일으킬 수 있으며, 신뢰와 소통이 중요한 연애에서 위기를 초래할 수 있습니다. 

2. 행동계획:
   - 먼저, 여자친구와의 연락이 소홀했던 이유를 명확히 이해하고, 그녀의 감정을 진정으로 존중하는 자세를 가집니다.
   - 연락하지 못했던 시간에 대해 사과의 메시지를 보내세요. 이때 변명보다는 솔직한 마음을 전하는 것이 중요합니다.
   - 그녀와의 대화를 통해 그녀가 느끼는 감정과 불만을 경청하고, 이를 통해 신뢰를 회복할 기회를 만드세요.
   - 이 상황을 해결하기 위한 적극적인 행동을 계획합니다. 즉, 그녀와의 특별한 데이트나 대화를 위한 시간을 예약하세요.

3. 실행:
   - 즉시 메시지를 보내 아래와 같은 내용을 포함시켜 보세요:
     "안녕, 어제 연락을 잘 못해서 미안해. 친구들이랑 있었던 것에 집중하느라 너와의 연락을 소홀히 했어. 네가 불편했을 것 같아 정말 미안해. 나한테 너의 기분을 이야기해줄 수 있을까? 그리고 가능하다면 이번 주말에 우리 같이 좋은 시간 보내고 싶어."
   - 그녀의 반응에 따라, 감정을 진정으로 수용하고 필요한 조치를 취하도록 합니다. 그녀가 원하는 대화나 데이트 제안에 맞춰 Plans를 잡아보세요. 
   - 이런 사소한 실수에서도 연습하여 더 나은 소통을 이어가는 데 포커스를 맞추세요.


## 냉털마스터 ReAct
- 사용자는 냉장고에 남아있는 음식재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지 조언해준다.
- Reasoing/Action을 끌어낼 수 있는 적절한 프롬프팅을 작성하세요.

In [34]:
def fridge_raid_master(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
  당신은 사용자가 입력한 냉장고 속 재료를 분석하여 창의적이고 실용적인 요리법을 추천하는 AI 요리사입니다.
  최적의 레시피를 제안하기 위해 아래 3단계로 답하세요.

  1. 상황분석
  입력받은 재료의 조합, 특징, 보관 상태, 영양 균형 등을 분석하고 어떤 종류의 요리를 만들 수 있을지 간략히 설명하세요.

  2. 행동계획
  가능한 요리 후보 중 하나를 선정하고, 그 선택의 이유를 논리적으로 제시하세요. 대체 재료나 보완재료(예: 기본양념, 추가하면 좋은 재료)도 함께 안내하세요.

  3. 실행
  추천한 요리의 레시피를 간단하고 명확한 단계로 안내하세요. (1, 2, 3 순서로 단계별로 적기)

  답변 시 친절함, 실용성, 그리고 조리의 편리함을 고려해 주십시오.
  필요하다면 조리 팁, 추가적으로 고려해야 할 내용도 안내하십시오.
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [35]:
from IPython.display import Markdown
user_foods = ['소고기', '양파', '표고버섯', '명란젓']

냉장고에 있는 재료로 몇 가지 요리를 제안해볼게요!

1. **삼겹살 구이**: 삼겹살을 간단하게 구워서 상추나 오이와 함께 쌈을 만들어 먹을 수 있어요.

2. **비빔면**: 비빔면을 삶아서 오이와 골뱅이를 얹고 고추장이나 참기름으로 간을 맞춰 비벼 먹는 것도 좋겠네요.

3. **골뱅이 무침**: 골뱅이를 오이와 함께 무쳐서 간단한 반찬으로 즐길 수 있어요. 고추장, 식초, 다진 마늘 등을 섞어 양념하시면 맛있습니다.

4. **삼겹살과 비빔면 조합**: 삼겹살을 구운 후 비빔면과 함께 서빙하면 서로 다른 맛의 조화가 잘 어울려요.

이 중에서 마음에 드는 요리를 선택해서 만들어보세요!


## 면접질문 생성 JSON

In [37]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
  당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로서, 해당분야의 으뜸가는 면접관입니다.
  매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

  <<지시사항>>
   - 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
   - 하드슨킬과 소프트스킬 두 개의 섹션으로 나누어 작성해주세요.
   - 각 스킬별로 질문/답변 3개씩 만들어주세요.
  <<출력형식>>
  출력은 json형식으로만 반환되어야 합니다.

"""
    user_message = f"""
채용공고:
{job_posting}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object" #💛json형식으로 받으려면 이렇게 지정해야됨
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [42]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""
output = job_interview(job_posting)
print(type(output))
print(output)

<class 'str'>
{
  "hard_skill": [
    {
      "question": "AI 모델의 배포를 위한 인프라 구축 시 고려해야 할 중요한 요소는 무엇인가요?",
      "answer": "AI 모델을 배포하기 위한 인프라 구축 시, 내가 주로 고려하는 요소는 확장성, 데이터 보안, 실제 운영 환경과의 호환성입니다. 확장성은 모델이 성장함에 따라 인프라를 자동으로 확장할 수 있어야 하며, 데이터 보안은 민감한 정보를 처리할 때 중요한 요소입니다. 또한 운영 환경과의 호환성을 통해 모든 시스템이 원활하게 상호작용할 수 있는지 확인해야 합니다."
    },
    {
      "question": "컨테이너 오케스트레이션 툴(Docker, Kubernetes 등)을 사용하는 과정에서의 경험을 공유해주시겠어요?",
      "answer": "이전 프로젝트에서 Kubernetes를 사용하여 마이크로서비스 아키텍처를 구축한 경험이 있습니다. 서비스 별로 서로 독립적으로 배포할 수 있어 유연성과 안정성을 높였습니다. Kubernetes의 Helm을 사용하여 배포 파이프라인을 자동화했고, 이를 통해 CI/CD의 효율성을 크게 향상시켰습니다."
    },
    {
      "question": "MLflow를 사용하여 모델 서빙 환경을 구축한 경험이 있나요? 구체적인 사례를 말씀해 주세요.",
      "answer": "네, MLflow를 사용하여 모델 서빙 환경을 구축한 경험이 있습니다. 특정 프로젝트에서 MLflow Tracking을 통해 다양한 실험 결과를 관리하고, MLflow Models를 통해 모델을 쉽게 배포할 수 있었습니다. 이를 통해 모델의 배포 주기를 단축시키고, 각 배포 버전의 성능을 지속적으로 모니터링했습니다."
    }
  ],
  "soft_skill": [
    {
      "question": "AI 인프라 팀 내에서 다른 팀원과의 협업을 어떻게 효과적으로 진행하시나요?",
      "answ

In [45]:
import json

data = json.loads(output)
print(type(data))
print(data)

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']
print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
{'hard_skill': [{'question': 'AI 모델의 배포를 위한 인프라 구축 시 고려해야 할 중요한 요소는 무엇인가요?', 'answer': 'AI 모델을 배포하기 위한 인프라 구축 시, 내가 주로 고려하는 요소는 확장성, 데이터 보안, 실제 운영 환경과의 호환성입니다. 확장성은 모델이 성장함에 따라 인프라를 자동으로 확장할 수 있어야 하며, 데이터 보안은 민감한 정보를 처리할 때 중요한 요소입니다. 또한 운영 환경과의 호환성을 통해 모든 시스템이 원활하게 상호작용할 수 있는지 확인해야 합니다.'}, {'question': '컨테이너 오케스트레이션 툴(Docker, Kubernetes 등)을 사용하는 과정에서의 경험을 공유해주시겠어요?', 'answer': '이전 프로젝트에서 Kubernetes를 사용하여 마이크로서비스 아키텍처를 구축한 경험이 있습니다. 서비스 별로 서로 독립적으로 배포할 수 있어 유연성과 안정성을 높였습니다. Kubernetes의 Helm을 사용하여 배포 파이프라인을 자동화했고, 이를 통해 CI/CD의 효율성을 크게 향상시켰습니다.'}, {'question': 'MLflow를 사용하여 모델 서빙 환경을 구축한 경험이 있나요? 구체적인 사례를 말씀해 주세요.', 'answer': '네, MLflow를 사용하여 모델 서빙 환경을 구축한 경험이 있습니다. 특정 프로젝트에서 MLflow Tracking을 통해 다양한 실험 결과를 관리하고, MLflow Models를 통해 모델을 쉽게 배포할 수 있었습니다. 이를 통해 모델의 배포 주기를 단축시키고, 각 배포 버전의 성능을 지속적으로 모니터링했습니다.'}], 'soft_skill': [{'question': 'AI 인프라 팀 내에서 다른 팀원과의 협업을 어떻게 효과적으로 진행하시나요?', 'answer': '효과적인 협업을 위해 주기적으로 의견을 교환하는 회의를 설정합니다. 또한, 각 팀원들이 가지고 있는 전문 지식과 경험을 바탕으로 업무를 분담하고, 필요 시